In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\IT STORE\\Desktop\\ML-Projects\\Text-Summarizer'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransfomationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [3]:
from src.constants import *
from src.utils import read_yaml, create_directories


In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
        
    def get_data_transformation_config(self) -> DataTransfomationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransfomationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name,
        )
        return data_transformation_config
        

In [5]:
import os 
from src.logger import logging
from transformers import BartTokenizer
from datasets import load_dataset, load_from_disk


In [12]:
class DataTransformation:
    def __init__(self,config: DataTransfomationConfig):

        self.config = config
        self.tokenizer = BartTokenizer.from_pretrained(config.tokenizer_name)
        
    def convert_examples_to_features(self, example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'], max_length=1024, truncate=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)
            
        return {
            'input_ids': input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
            'decoder_input_ids': target_encoding['input_ids'],
            'decoder_attention_mask': target_encoding['attention_mask'],
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [ ]:
import os
from datasets import load_from_disk

try:
    dataset_path = os.path.join(self.config.root_dir, "samsum_dataset")
    dataset_samsum = load_from_disk(dataset_path)
except ValueError as e:
    print(f"Error loading dataset: {e}")
    print(f"Dataset path: {dataset_path}")
    print("Check if the dataset is saved correctly and the path is correct.")

In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

FileNotFoundError: Directory artifacts/data_ingestion is neither a `Dataset` directory nor a `DatasetDict` directory.